In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip \
    -O /tmp/horse-or-human.zip

--2023-01-09 16:55:31--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.194.128, 74.125.68.128, 74.125.24.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.194.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149574867 (143M) [application/zip]
Saving to: ‘/tmp/horse-or-human.zip’

/tmp/horse-or-human 100%[===================>] 142.65M  24.0MB/s    in 7.0s    

2023-01-09 16:55:39 (20.3 MB/s) - ‘/tmp/horse-or-human.zip’ saved [149574867/149574867]



In [ ]:
import os, zipfile

In [ ]:
local_zip = "/tmp/horse-or-human.zip"

In [ ]:
zip_ref = zipfile.ZipFile(local_zip, 'r')

In [ ]:
zip_ref.extractall('/tmp/horse-or-human/')

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(
    rescale = 1/255.0,
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = 'nearest'
)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    '/tmp/horse-or-human',
    target_size = (224, 224),
    batch_size = 128,
    class_mode = 'binary'
)

Found 1027 images belonging to 2 classes.


In [ ]:
train_generator.class_indices

{'horses': 0, 'humans': 1}

In [ ]:
import tensorflow

In [ ]:
base_model = tensorflow.keras.applications.mobilenet_v2.MobileNetV2(
    input_shape = (224, 224, 3),
    include_top = False,
    weights = 'imagenet'
)

9406464/9406464 [==============================] - 0s 0us/step


In [ ]:
base_model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                              

In [ ]:
for layer in base_model.layers:
  layer.trainable = False

In [ ]:
last_layer = base_model.layers[-1]

In [ ]:
from tensorflow.keras.layers import Flatten, Dense

In [ ]:
x = Flatten()(last_layer.output)
x = Dense(1024, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)

In [ ]:
model = tensorflow.keras.models.Model(base_model.input, x)

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                             

In [ ]:
model.compile(loss = 'binary_crossentropy', 
              optimizer=tensorflow.keras.optimizers.RMSprop(lr = 0.001), metrics = 'accuracy')

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=8,
    epochs=15,
    verbose=1
)

Epoch 1/15
8/8 [==============================] - 24s 2s/step - loss: 80.0011 - accuracy: 0.7588
Epoch 2/15
8/8 [==============================] - 14s 2s/step - loss: 0.1253 - accuracy: 0.9933
Epoch 3/15
8/8 [==============================] - 13s 2s/step - loss: 0.1910 - accuracy: 0.9889
Epoch 4/15
8/8 [==============================] - 13s 2s/step - loss: 3.3237 - accuracy: 0.9088
Epoch 5/15
8/8 [==============================] - 13s 2s/step - loss: 8.4666e-05 - accuracy: 1.0000
Epoch 6/15
8/8 [==============================] - 13s 2s/step - loss: 0.6910 - accuracy: 0.9789
Epoch 7/15
8/8 [==============================] - 13s 2s/step - loss: 0.1175 - accuracy: 0.9944
Epoch 8/15
8/8 [==============================] - 13s 2s/step - loss: 0.1914 - accuracy: 0.9844
Epoch 9/15
8/8 [==============================] - 15s 2s/step - loss: 0.0068 - accuracy: 0.9990
Epoch 10/15
8/8 [==============================] - 13s 2s/step - loss: 0.0556 - accuracy: 0.9956
Epoch 11/15
8/8 [=================

In [ ]:


import numpy as np
from google.colab import files
from tensorflow.keras.utils import load_img, img_to_array
     


In [ ]:

uploaded = files.upload()

for fn in uploaded.keys():
  path = '/content/' + fn
  img = load_img(path, target_size=(224, 224))
  x = img_to_array(img)
  x = np.expand_dims(x, axis=0)

  image_tensor = np.vstack([x])
  classes = model.predict(image_tensor)
  print(classes[0])
     


Saving images.jpeg to images.jpeg
1/1 [==============================] - 1s 952ms/step
[1.4381037e-05]


In [ ]:
classes

array([[1.4381037e-05]], dtype=float32)

In [ ]:
train_generator.class_indices

{'horses': 0, 'humans': 1}

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [ ]:
weights_url

In [ ]:
weights_url = "https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"

In [ ]:
import urllib

In [ ]:
urllib.request.urlretrieve(weights_url, "inception_v3.h5")

('inception_v3.h5', <http.client.HTTPMessage at 0x7f0e83607af0>)

In [ ]:
pre_trained_model = InceptionV3(
    input_shape=(224, 224, 3),
    include_top = False,
    weights=None
)

In [ ]:
pre_trained_model.load_weights('inception_v3.h5')

In [ ]:
pre_trained_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 111, 111, 32  864         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 111, 111, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                      

In [ ]:
for layer in pre_trained_model.layers:
  layer.trainable = False

In [ ]:
last_layer = pre_trained_model.get_layer('mixed7')

In [ ]:
import tensorflow
from tensorflow.keras.layers import Flatten, Dense

In [ ]:
x = Flatten()(last_layer.output)
x = Dense(1024, activation = 'relu')(x)
x = Dense(1, activation = 'sigmoid')(x)

model = tensorflow.keras.models.Model(pre_trained_model.input, x)

In [ ]:
model.compile(loss = 'binary_crossentropy', 
              optimizer=tensorflow.keras.optimizers.RMSprop(lr = 0.001), metrics = 'accuracy')

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=8,
    epochs=15,
    verbose=1
)

Epoch 1/15
8/8 [==============================] - 18s 2s/step - loss: 14.8915 - accuracy: 0.6329
Epoch 2/15
8/8 [==============================] - 14s 2s/step - loss: 0.0710 - accuracy: 0.9900
Epoch 3/15
8/8 [==============================] - 15s 2s/step - loss: 0.0175 - accuracy: 0.9933
Epoch 4/15
8/8 [==============================] - 14s 2s/step - loss: 0.0090 - accuracy: 0.9967
Epoch 5/15
8/8 [==============================] - 14s 2s/step - loss: 0.0157 - accuracy: 0.9922
Epoch 6/15
8/8 [==============================] - 14s 2s/step - loss: 0.0308 - accuracy: 0.9922
Epoch 7/15
8/8 [==============================] - 14s 2s/step - loss: 0.0705 - accuracy: 0.9867
Epoch 8/15
8/8 [==============================] - 14s 2s/step - loss: 1.5457 - accuracy: 0.9055
Epoch 9/15
8/8 [==============================] - 14s 2s/step - loss: 0.0972 - accuracy: 0.9878
Epoch 10/15
8/8 [==============================] - 14s 2s/step - loss: 0.0178 - accuracy: 0.9978
Epoch 11/15
8/8 [=====================

In [ ]:
model.trainable_weights

[<tf.Variable 'dense_2/kernel:0' shape=(110592, 1024) dtype=float32, numpy=
 array([[-0.00923952, -0.00233022, -0.00828039, ..., -0.00514045,
          0.00011582, -0.0076954 ],
        [-0.00771541, -0.00259927, -0.00907671, ..., -0.00180125,
         -0.00595433, -0.0020702 ],
        [-0.00100501,  0.00117324,  0.00250987, ...,  0.00185286,
         -0.00374214,  0.00093514],
        ...,
        [ 0.00358978, -0.00531959,  0.00775438, ..., -0.00404451,
         -0.01017302, -0.0017989 ],
        [-0.00613249,  0.00248215, -0.00621469, ..., -0.00409411,
         -0.00073312, -0.00758682],
        [-0.00290673, -0.00309249, -0.0066517 , ...,  0.00524606,
         -0.01018194, -0.00596327]], dtype=float32)>,
 <tf.Variable 'dense_2/bias:0' shape=(1024,) dtype=float32, numpy=
 array([-3.1610865e-03,  3.6323920e-05, -3.1621403e-03, ...,
        -3.1621004e-03,  2.3304718e-05, -3.1611330e-03], dtype=float32)>,
 <tf.Variable 'dense_3/kernel:0' shape=(1024, 1) dtype=float32, numpy=
 array([